In [1]:
# Import libraries
from gurobipy import*

import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
# File path 
FILE = "./DaSilvaUrrutia/n200w100.001.txt"

In [3]:
# Default params
SUPPORTED_FORMAT = ['NUM', 'X', 'Y', 'DEMAND', 'READYTIME', 'DUEDATE', 'SERVICE']
MINUTES = 60
HOURS = 3600
OFFSET_TIMES = 8*HOURS

COLUMNS_OPS = {'NUM': lambda x: float(x),
               'X': lambda x: float(x),
               'Y': lambda x: float(x),
               'DEMAND': lambda x: 1,
               #'READYTIME': lambda x: float(x)+OFFSET_TIMES,
               'READYTIME': lambda x: float(x),
               #'DUEDATE': lambda x: float(x)+OFFSET_TIMES,
               'DUEDATE': lambda x: float(x),

               'SERVICE': lambda x: 30*MINUTES
              }

AGENTS = 10

TIME_PER_DISTANCE = 1

#WORKING_TIME_RANGE = (OFFSET_TIMES, OFFSET_TIMES + 8*HOURS)
WORKING_TIME_RANGE = (0, 8*HOURS)

#LUNCH_BREAK_RANGE = (12*HOURS, 13.5*HOURS)
LUNCH_BREAK_RANGE = (12*HOURS-OFFSET_TIMES, 13.5*HOURS-OFFSET_TIMES)
LUNCH_BREAK_TIME = 30*MINUTES

OFFICE_NUM = 0
OFFICE_X = .0
OFFICE_Y = .0
OFFICE_READYTIME = WORKING_TIME_RANGE[0]
OFFICE_DUEDATE = WORKING_TIME_RANGE[1]
OFFICE_SERVICE = 1*HOURS

In [4]:
LUNCH_BREAK_RANGE

(14400, 19800.0)

In [5]:
WORKING_TIME_RANGE

(0, 28800)

In [6]:
def read_input_tsptw(filename):
    """This function is used to convert input file to usable data"""
    nb_nodes = 0
    
    data_dict = dict()
    
    nodes_x = list()
    nodes_y = list()
       
    # Open file and read lines 
    with open(filename, "r") as file:
        # Initialize columns in empty dict
        columns = file.readline().replace("#","").split()
        if columns != SUPPORTED_FORMAT:
            print("ERROR! Format not supported.")
            return 
        
        # Add office to data
        data_dict.update({OFFICE_NUM: {'X': OFFICE_X, 
                                       'Y': OFFICE_Y, 
                                       'DEMAND': AGENTS,
                                       'READYTIME': OFFICE_READYTIME,
                                       'DUEDATE': OFFICE_DUEDATE,
                                       'SERVICE': OFFICE_SERVICE,}})
        # Add office to nodes
        nodes_x.append(OFFICE_X)
        nodes_y.append(OFFICE_Y)
            
        # For each data line
        for line in file.readlines():
            node_dict = {k: COLUMNS_OPS[k](val) for k, val in zip(columns, line.split())}
                
            # Get id
            node_id = node_dict.pop('NUM')
            # Insert new node in data dict
            data_dict.update({int(node_id): node_dict})            
            # Get nodes positions
            nodes_x.append(float(line.split()[columns.index('X')]))
            nodes_y.append(float(line.split()[columns.index('Y')]))

    # Get distance matrix
    distance_matrix = compute_distance_matrix(nodes_x, nodes_y)
    
    return (data_dict, distance_matrix, dict(enumerate(zip(nodes_x, nodes_y))))


def compute_distance_matrix(nodes_x, nodes_y):
    """This function is used to compute the distance matrix"""
    nb_customers = len(nodes_x)
    distance_matrix = [[None for i in range(nb_customers)] for j in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(nodes_x[i], nodes_x[j], nodes_y[i], nodes_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


def compute_dist(xi, xj, yi, yj):
    """This function is used to compute euclidean distance"""
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5)) * TIME_PER_DISTANCE

In [7]:
# Getting parameters
data_dict, distance_matrix, positions = read_input_tsptw(FILE)

In [8]:
len(data_dict)

202

In [9]:
# DEBUG RESTRICTIONS
CLIENTS = 25
data_dict = {k: v for k,v in data_dict.items() if k < CLIENTS}
distance_matrix = [dm[:CLIENTS] for dm in distance_matrix[:CLIENTS]]

# ADD FITTICIOUS POSITION
distance_matrix = [dm + [0,] for dm in distance_matrix]
distance_matrix = distance_matrix + [[0]*(CLIENTS+1)]
# Add office to data
data_dict.update({CLIENTS: {'X': 0, 
                            'Y': 0, 
                            'DEMAND': AGENTS,
                            'READYTIME': WORKING_TIME_RANGE[0],
                            'DUEDATE': WORKING_TIME_RANGE[1],
                            'SERVICE': 0,}})

# POSITIONS SETS
all_pos = list(range(CLIENTS*2))
start_pos = CLIENTS
client_pos = list(range(1,CLIENTS))
only_start_office_pos = 0
no_duplicates_pos = list(range(0,CLIENTS))
destination_office_pos = list(range(CLIENTS+1, CLIENTS*2))
office_pos = [only_start_office_pos]+destination_office_pos
reachable_pos = [p for p in range(CLIENTS*2) if p not in [start_pos,]]

In [10]:
# Create model
mod = Model("TSPTW")    

Using license file c:\gurobi903\gurobi.lic
Academic license - for non-commercial use only


In [11]:
# IMPORTANT! -> Office IS ONE AND ONLY ONE but because i wasn't unable to find a better solution for 
#               multiple Agent visit to it i've repeated Office (originally in position 0) in all 
#               position greater than number_of_clients + 1
# WARNING! -> Position from number_of_clients + 2 forward are office positions reachable only from 
#             player = office_pos - number_of_clients
# WARNING! -> Position number_of_clients + 1 is unused because position 0 is office too, Agents can
#             only start from office 0 they can't go there

In [12]:
# VARIABLES
# Served client
x = mod.addVars({(i,a): 0 for i in all_pos
                          for a in range(AGENTS)},
               name="x",
               vtype=GRB.BINARY)

#  Agent trip
y = mod.addVars({(i,j,a): 0 for i in all_pos
                            for j in all_pos 
                            for a in range(AGENTS)}, 
                name="y", 
                vtype=GRB.BINARY)

# Serve Order
o = mod.addVars({(i,a): 0 for i in all_pos
                          for a in range(AGENTS)},
               name="o",
               vtype=GRB.INTEGER)

# Serve time
s =  mod.addVars({(i,a): 0 for i in all_pos 
                           for a in range(AGENTS)}, 
                 name="s", 
                 vtype=GRB.INTEGER)

# Serve Client/Office lasting
c = mod.addVars({(i,j,a): 0 for i in all_pos
                            for j in all_pos
                            for a in range(AGENTS)}, 
                name="c", 
                vtype=GRB.INTEGER)

# Wait time
w = mod.addVars({(i,a): 0 for i in all_pos
                          for a in range(AGENTS)}, 
                name="w", 
                vtype=GRB.INTEGER)

# Lunch done between customers
l = mod.addVars({(i,j,a): 0 for i in all_pos
                            for j in all_pos
                            for a in range(AGENTS)}, 
                name="l", 
                vtype=GRB.INTEGER)

# Is lunch to do
t = mod.addVars({(a): 0 for a in range(AGENTS)},
               name="t",
               vtype=GRB.INTEGER)

In [13]:
# CONSTRAINTS

# All client must be visited from an Agent
_= mod.addConstrs((quicksum(x[i,a]
                            for a in range(AGENTS)) == 1
                  for i in client_pos),
                  name="ServeAll")

# All duplicated Offices must be visited from at most an Agent
_= mod.addConstrs((quicksum(x[i,a]
                            for a in range(AGENTS)) <= 1
                  for i in destination_office_pos),
                  name="ServeDuplicatesOffice")

# All Agents must have fitticiuous position in their trip
_= mod.addConstrs((x[start_pos,a] == 1
                   for a in range(AGENTS)),
                  name="HasFitticious")

# All Agents start their trip from fitticious position
_= mod.addConstrs((quicksum(y[start_pos,j,a] 
                            for j in reachable_pos) == 1
                  for a in range(AGENTS)),
                  name="StartFromFitticious")

# Agents can't start their trip from fitticious position and go to only destination Office
_= mod.addConstrs((y[start_pos,j,a] == 0
                  for j in destination_office_pos
                  for a in range(AGENTS)),
                  name="NotStartFromFitticiousToOffice")
                  
# All Agents end their trip in fitticious position 
_= mod.addConstrs((quicksum(y[i,start_pos,a] 
                            for i in reachable_pos) == 1
                  for a in range(AGENTS)),
                  name="EndInFitticious")
                  
# Agent visit Client once and only if he serve him
_= mod.addConstrs((quicksum(y[i,j,a]
                            for i in all_pos if i != j
                           ) <= x[j,a] 
                  for a in range(AGENTS)
                  for j in reachable_pos),
                  name="ServeOnce")

_= mod.addConstrs((quicksum(y[i,h,a] for i in all_pos) - 
                   quicksum(y[h,j,a] for j in all_pos) == 0
                  for h in all_pos
                  for a in range(AGENTS)),
                  name="ContinuousLoops")

# Agent can't do loop between same Client
_= mod.addConstrs((y[i,i,a] == 0
                  for a in range(AGENTS)
                  for i in all_pos),
                  name="NoSelfLoops")

# Sum of time spent into a position in a specific trip
_= mod.addConstrs((quicksum(c[i,j,a]*y[i,j,a] for i in all_pos) == x[j,a] * data_dict[j]['SERVICE']
                  for j in client_pos
                  for a in range(AGENTS)),
                  name="ServingTime"
                 )

# Sum of time spent into a office in a specific trip
_= mod.addConstrs((quicksum(c[i,j,a]*y[i,j,a] 
                          for i in all_pos
                          for j in office_pos) == data_dict[0]['SERVICE']
                  for a in range(AGENTS)),
                  name="ServingTimeOffice"
                 )

# Can spent time in a position only if it's in the trip
_= mod.addConstrs((c[i,j,a] <= y[i,j,a] * data_dict[j if j in no_duplicates_pos else 0]['SERVICE'] 
                   for i in reachable_pos
                   for j in reachable_pos
                   for a in range(AGENTS)),
                  name="ServingOnlyIfInTrip"
                 )

# Serving order for each Agent served Client is calculated as 
# (is actual Client served) + (serving order of Client before him)
_= mod.addConstrs((o[i,a] == x[i,a]+quicksum(o[j,a]*y[j,i,a] for j in reachable_pos if i!=j) 
                  for a in range(AGENTS)
                  for i in reachable_pos),
                  name="ServeOrder")

# Serving order check is calculated as:
# (sum of all serving values) must be equal to (served Clients)*(served Clients + 1) / 2
_= mod.addConstrs((quicksum(o[i,a] for i in reachable_pos) == quicksum(x[i,a] 
                                                            for i in reachable_pos)*
                                                            (quicksum(x[i,a] for i in reachable_pos)+1)/2
                  for a in range(AGENTS)),
                  name="CheckServeOrder")

# Sum of minutes spent in: travels, servicing clients, waiting, eating at lunch 
_= mod.addConstrs((quicksum(y[i,j,a] * distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0]
                           for i in reachable_pos
                           for j in reachable_pos) +
                   quicksum(x[i,a] * data_dict[i if i in no_duplicates_pos else 0]['SERVICE']
                           for i in reachable_pos) +
                   quicksum(w[i,a]
                           for i in reachable_pos) +
                   WORKING_TIME_RANGE[0] + 
                   quicksum(l[i,j,a]
                            for i in reachable_pos
                            for j in reachable_pos) * LUNCH_BREAK_TIME <= WORKING_TIME_RANGE[1]
                   for a in range(AGENTS)),
                  name="MaxHours")

In [14]:
# Agent must go office at least once or start from it [0]
_= mod.addConstrs((quicksum(x[i,a] for i in office_pos) >= 1 
                            for a in range(AGENTS)),
                  name="ServeOffice")

# Agent can't go office from office
_= mod.addConstrs((y[i,j,a] == 0  
                            for i in office_pos
                            for j in office_pos
                            for a in range(AGENTS)),
                  name="NoOfficeFromOfficeB")

# That is an only start Office position
_= mod.addConstrs((y[i,0,a] == 0  
                            for i in client_pos
                            for a in range(AGENTS)),
                  name="NoOfficeFromOfficeC")

# Agent can go to Office only if he visited the associated Client 
_= mod.addConstrs((x[i+CLIENTS,a] <= x[i,a]
                   for i in client_pos
                   for a in range(AGENTS)),
                  name="OfficeServedOnlyIfItsClientIs")

# Agent can go to Office just after he visit the associated Client [PROBLEMA]
_= mod.addConstrs((x[i+CLIENTS,a] <= y[i,i+CLIENTS,a]
                            for i in client_pos
                  for a in range(AGENTS)),
                  name="GoOfficeAfterItsClient")

In [15]:
# Service time of Client J is equal to sum of:
# wait_time_of_client_J, service_time_of_client_I_before_of_J, trip_between_I_and_J, 
# time_of_service_of_client_I, lunch_time_if_present
_= mod.addConstrs((s[j,a] == quicksum(y[i,j,a]*s[i,a] + 
                                      y[i,j,a]*distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0] +
                                      y[i,j,a]*l[i,j,a]*LUNCH_BREAK_TIME
                                      for i in reachable_pos if i!=j) +
                   x[j,a]*w[j,a]+
                   quicksum(y[i,j,a]*c[h,i,a] 
                            for i in reachable_pos if i!=j
                            for h in reachable_pos if h!=i)
                    for a in range(AGENTS)
                    for j in reachable_pos), 
                  name="RouteInTime")

# _= mod.addConstrs((y[i,j,a] <= c[i,j,a] for i in all_pos for j in reachable_pos for a in range(AGENTS)),
#                  name="GoOnlyIfNeeded")

# Agent serve Client after his time window start
_= mod.addConstrs((data_dict[i]['READYTIME']*x[i,a] <= s[i,a]
                   for i in client_pos
                   for a in range(AGENTS)),
                  name="ServeAfterTWStart")

# Agent serve Client before his time window end
_= mod.addConstrs((data_dict[i]['DUEDATE']*x[i,a] >= s[i,a]
                   for i in client_pos
                   for a in range(AGENTS)),
                  name="ServeBeforeTWEnd")

# If Agent have lunch between Clients I and J, agent have to make trip between I and J  
_= mod.addConstrs((l[i,j,a] <= y[i,j,a]
                   for i in reachable_pos
                   for j in reachable_pos
                   for a in range(AGENTS)),
                  name="LunchTime")

# If Agent has lunch it must be after lunch time start
_= mod.addConstrs((l[i,j,a]*(s[i,a] + 
                             c[i,j,a] + 
                             distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0] + 
                             w[j,a]) >= l[i,j,a]*LUNCH_BREAK_RANGE[0] 
                   for i in reachable_pos
                   for j in reachable_pos
                   for a in range(AGENTS)),
                  name="LunchTimeStart")

# If Agent has lunch it must be before lunch time end
_= mod.addConstrs((l[i,j,a]*(s[i,a] + 
                             c[i,j,a] +
                             distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0] +
                             w[j,a]) <= l[i,j,a]*LUNCH_BREAK_RANGE[1] 
                   for i in reachable_pos
                   for j in reachable_pos
                   for a in range(AGENTS)),
                  name="LunchTimeEnd")

# Getting maximum Agent service time
_= mod.addConstrs((t[a] == max_(s[i,a] for i in reachable_pos) 
                   for a in range(AGENTS)),
                  name="MaxServiceTime")

# If Agent working time is greater than lunch time than Agent must have lunch
_= mod.addConstrs((WORKING_TIME_RANGE[1] * (quicksum(l[i,j,a]
                                             for i in reachable_pos
                                             for j in reachable_pos)) >= t[a] - LUNCH_BREAK_RANGE[0]
                   for a in range(AGENTS)),
                  name="AgentNeedLunchI")

# If Agent working time is lesser than lunch time than Agent must not have lunch
_= mod.addConstrs((WORKING_TIME_RANGE[1] * (1 - quicksum(l[i,j,a]
                                             for i in reachable_pos
                                             for j in reachable_pos)) >= LUNCH_BREAK_RANGE[0] - t[a]
                   for a in range(AGENTS)),
                  name="AgentNeedLunchII")

In [16]:
mod.setObjective(quicksum(y[i,j,a]*distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0]
                  for i in all_pos
                  for j in all_pos
                  for a in range(AGENTS)) + 
                 quicksum(w[i,a]
                  for i in reachable_pos
                  for a in range(AGENTS)), 
                 GRB.MINIMIZE)

In [ ]:
mod.params.Method=0
mod.params.TimeLimit=2000
#mod.params.SolutionLimit=2
mod.optimize()

Parameter Method unchanged
   Value: 0  Min: -1  Max: 5  Default: -1
Parameter TimeLimit unchanged
   Value: 2000.0  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 57318 rows, 77010 columns and 270950 nonzeros
Model fingerprint: 0x55aa8be2
Model has 49260 quadratic constraints
Model has 10 general constraints
Variable types: 0 continuous, 77010 integer (25500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  QMatrix range    [5e-01, 2e+03]
  QLMatrix range   [5e-01, 2e+04]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
  QRHS range       [4e+03, 4e+03]
Presolved: 1145108 rows, 404230 columns, 3127810 nonzeros
Presolved model has 990 SOS constraint(s)

Continuing optimization...



In [18]:
#mod.computeIIS()

In [19]:
from copy import deepcopy
def domino(otrip):
    trip = deepcopy(otrip)
    sorted_trip = list()
    while len(trip):
        old = deepcopy(trip)
        # Find start
        for x in trip:
            not_start = False
            for y in trip:
                if x[0] == y[1]:
                    not_start = True
                    break
            if not not_start:
                sorted_trip.append(trip.pop(trip.index(x)))
                break
        # Continue dominos trip
        next_found = True
        while next_found:
            next_found = False
            for x in trip:
                if sorted_trip[-1][1] == x[0]:
                    sorted_trip.append(trip.pop(trip.index(x)))
                    next_found = True
                    break
                    
        if trip == old:
            for t in trip:
                sorted_trip.append(trip.pop(trip.index(x)))
    return sorted_trip

for a in range(AGENTS):
    agent_trip = list()
    for i in all_pos:
        for j in all_pos:
            if y[i,j,a].X:
                agent_trip.append((i if i != start_pos else 'START',j if j != start_pos else 'END', 
                                   s[j,a].X, 
                                   c[i,j,a].X,
                                   s[j,a].X+c[i,j,a].X))
                
    # Find start
    print(f"Agent {a}: {agent_trip}")

# PERCORSI DI VISITA

AttributeError: Unable to retrieve attribute 'X'

In [ ]:
# ORDINE DI VISITA (ordine di visita, cliente visitato)
for a in range(AGENTS):
    agent_clients = list()
    for i in all_pos:
        if o[i,a].X:
            agent_clients.append((o[i,a].X, i))
    # Find start
    print(f"Agent {a}: {sorted(agent_clients, key=lambda tup: tup[0])}")

In [ ]:
# MAX TEMPO VISITA
for a in range(AGENTS):
    # Find start
    print(f"Agent {a}: {t[a].X}")

In [ ]:
# CHE CLIENTI VISITA OGNI AGENTE
for a in range(AGENTS):
    agent_clients = list()
    for i in all_pos:
        if x[i,a].X:
            agent_clients.append(i)
    # Find start
    print(f"Agent {a}: {agent_clients}")

In [ ]:
# ATTESE DI VISITA E TEMPI DI VISITA
for a in range(AGENTS):
    agent_clients = list()
    for i in all_pos:
        if w[i,a].X:
            agent_clients.append((i, w[i,a].X, s[i,a].X, data_dict[i if i in no_duplicates_pos else 0]["READYTIME"]))
    # Find start
    print(f"Agent {a}: {agent_clients}")

In [ ]:
# ATTESE DI VISITA E TEMPI DI VISITA
for a in range(AGENTS):
    agent_clients = list()
    if t[a].X:
        # Find start
        print(f"Agent {a} have lunch.({sum([l[i,j,a].X for i in all_pos for j in all_pos])})")

In [ ]:
LUNCH_BREAK_RANGE[0]

In [ ]:
for a in range(AGENTS):
    print("AGENT ",a)
    for i in all_pos:
        for j in all_pos:
            if y[i,j,a].X:
                print("___________________")
                print(data_dict[i if i in no_duplicates_pos else 0]['SERVICE']+distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0])
                print(s[i,a].X+data_dict[i if i in no_duplicates_pos else 0]['SERVICE']+distance_matrix[i if i in no_duplicates_pos else 0][j if j in no_duplicates_pos else 0]+w[j,a].X)

In [ ]:
for a in range(AGENTS):
    for i in all_pos:
        for j in all_pos:
            if y[i,j,a].X:
                print("------")
                print(s[i,a].X)
                print(s[j,a].X)

### Define function to plot the graph

In [ ]:
def plot_model(model, positions):
    """This function is used to plot graph"""
    # Create figure
    plt.figure(figsize=(16,9))
    G = nx.Graph()
    # Add nodes to graph
    G.add_nodes_from(positions.keys())
    
    # Set labels dict
    node_labels = dict()
    # Set nodes positions
    for n, p in positions.items():
        G.nodes[n]['pos'] = p
        node_labels[n] = n

    # Define edges colors
    edge_colors = [plt.cm.tab20.colors[i] for i in model.keys()]
    # Add edges
    for t, l in model.items():
        for i in range(1,len(l['trip'])):
            G.add_edge(l['trip'][i-1], l['trip'][i], 
                       color=edge_colors[t], 
                       alpha=0.5, 
                       weight=4)
    # Set edges parameters 
    edges = G.edges()
    colors = [G[u][v]['color'] for u,v in edges]
    weights = [G[u][v]['weight'] for u,v in edges]
    alphas = [G[u][v]['alpha'] for u,v in edges]

    # Draw Nodes
    nx.draw_networkx_nodes(G, positions,
                           node_size=200,
                           node_color='c')
    # Draw Labels
    nx.draw_networkx_labels(G, positions, 
                            node_labels, 
                            font_size=10, 
                            font_color='k')
    # Draw Edges 
    nx.draw_networkx_edges(G, positions, 
                           alpha=0.5, 
                           width=weights, 
                           edge_color=colors)

    # Show graph
    plt.show()

# Create model
mod = Model("TSPTW")     